In [ ]:
import holoviews as hva
import panel as pn

hv.extension("bokeh") # DON'T KNOW WHY THIS WAS SUDDENLY NEEDED. PLEASE HELP
pn.extension('plotly') # We add 'plotly' to be able to use the 'plotly' plot backend

[HoloViews](https://holoviews.org/) is a very powerful data visualization library supporting many data and plotting *backends*.

[hvPlot](https://hvplot.holoviz.org/index.html) (quick viz) and [GeoViews](https://holoviz.org/assets/geoviews.png) (spatial viz) are built on top of HoloViews and produces `HoloViews` objects.

**Panel, HoloViews, hvPlot and GeoViews are all members of the [HoloViz](https://holoviz.org) ecosystem and you can expect them to work perfectly together**.

<img src="https://holoviz.org/assets/holoviews.png" style="height:50px;margin-right:10px"> <img src="https://holoviz.org/assets/hvplot.png" style="height:50px;margin-right:10px"> <img src="https://holoviz.org/assets/geoviews.png" style="height:50px;margin-right:10px"> <img src="https://holoviz.org/assets/panel.png" style="height:50px;margin-right:10px">

In this reference notebook we will assume a basic level of understanding of HoloViews and optionally hvPlot or Geoviews if you want to use them with Panel.

---

The `HoloViews` pane renders [HoloViews](https://holoviews.org/) objects with one of the *plotting backends* supported by HoloViews. This includes objects produced by [hvPlot](https://hvplot.holoviz.org/index.html) and [GeoViews](https://holoviz.org/assets/geoviews.png).

The `HoloViews` pane supports displaying interactive [`HoloMap`](https://holoviews.org/reference/containers/bokeh/HoloMap.html) and [`DynamicMap`](https://holoviews.org/reference/containers/bokeh/DynamicMap.html) objects containing widgets. The `HoloViews` pane even allows customizing the widget types and their position relative to the plot.

#### Parameters:

For details on other options for customizing the component see the [layout](../../how_to/layout/index.md) and [styling](../../how_to/styling/index.md) how-to guides.

The main argument is the `object` parameter

* **``object``** (object): The HoloViews object being displayed

You can control the way the `object` is rendered and sharing axes with other plots via the parameters

* **``backend``** (str): Any of the supported HoloViews backends ('bokeh', 'matplotlib', or 'plotly'). If none is specified defaults to the active holoviews renderer. This is by default 'bokeh'.
* **``linked_axes``** (boolean, default=True): Whether to link axes across plots in a panel layout
* **``renderer``**: Explicit [HoloViews Renderer](https://holoviews.org/user_guide/Plots_and_Renderers.html#renderers) instance to use for rendering the HoloViews  plot. Overrides the` backend` parameter.
* **``theme`` (str, Theme)**: [Bokeh theme](https://docs.bokeh.org/en/latest/docs/reference/themes.html) to apply to the HoloViews plot.

You can access the layout and (optional) widgets via

* **``layout``** (pn.layout.Panel): The layout containing the plot pane and (optionally) the `widget_box` layout.
* **``widget_box``** (ListPanel): The layout containing the widgets

##### Layout and Widget Parameters

The below parameters are used to control the layout and widgets using `pn.panel` or `pn.pane.HoloViews(...).layout`.

* **``center``** (boolean, default=False): Whether to center the plot or not.
* **``widgets``** (dict, argument): A mapping from dimension name to a widget class, instance, or dictionary of overrides to modify the default widgets. Provided as an argument.
* **``widget_location``** (str): Where to lay out the widget relative to the plot 
* **``widget_layout``** (ListPanel): The object to lay the widgets out in, one of ``Row``, ``Column`` or ``WidgetBox``
* **``widget_type``** (str): Whether to generate individual widgets for each dimension, or to use a global linear scrubber with dimensions concatenated.
___

The `pn.pane.HoloViews` pane will automatically convert any ``HoloViews`` object into a displayable panel, while keeping all of its interactive features:

In [ ]:
import numpy as np

data = {"group": np.random.randint(0, 10, 100), "value": np.random.randn(100)}
box = hv.Scatter(data, kdims="group", vdims="value").sort().opts()

hv_layout = pn.pane.HoloViews(box, height=300, sizing_mode="stretch_width")
hv_layout

By setting the pane's ``object`` the plot can be updated like all other pane objects:

In [ ]:
hv_layout.object = hv.Violin(box).opts(violin_color='Group', cmap='Category20', responsive=True, height=300)

You can display [`HoloMap`](https://holoviews.org/reference/containers/bokeh/HoloMap.html) and [`DynamicMap`](https://holoviews.org/reference/containers/bokeh/DynamicMap.html) too. 

[HoloViews](https://holoviews.org/) (the framework) natively renders plots with widgets if a [`HoloMap`](https://holoviews.org/reference/containers/bokeh/HoloMap.html) or [DynamicMap](https://holoviews.org/reference/containers/bokeh/DynamicMap.html) declares any *key dimensions*. This approach efficiently updates just the data inside a plot instead of replacing the plot entirely.

Please note that if you want to use the layout and widgets parameters, then you must use use `pn.panel` or `pn.pane.HoloViews(...).layout` instead of `pn.pane.HoloViews`. More info below.

In [ ]:
import pandas as pd
import hvplot.pandas
import holoviews.plotting.bokeh

def sine(frequency=1.0, amplitude=1.0, function='sin'):
    xs = np.arange(200)/200*20.0
    ys = amplitude*getattr(np, function)(frequency*xs)
    return pd.DataFrame(dict(y=ys), index=xs).hvplot(height=250, responsive=True)

dmap = hv.DynamicMap(sine, kdims=['frequency', 'amplitude', 'function']).redim.range(
    frequency=(0.1, 10), amplitude=(1, 10)).redim.values(function=['sin', 'cos', 'tan'])

hv_panel = pn.pane.HoloViews(dmap)
hv_panel

The `layout` parameter contains the `HoloViews` pane and the `widget_box`.

In [ ]:
print(hv_panel.layout, "\n\n", hv_panel.widget_box)

You can use [hvPlot](https://hvplot.holoviz.org/) (and [GeoViews](https://geoviews.org/)) too.

In [ ]:
import pandas as pd
import hvplot.pandas

df = pd.DataFrame(data)
plot = df.hvplot.box(by="group", y="value", responsive=True, height=300)
pn.pane.HoloViews(plot, height=300, sizing_mode="stretch_width")

## Backend

The ``HoloViews`` pane will default to the 'bokeh' plotting backend if no backend has been loaded via `holoviews`, but you can change the backend to any of the 'bokeh', 'matplotlib' and 'plotly' backends as needed.

### Bokeh

Bokeh is the default plotting backend, so normally you don't have to specify it. But lets do it here to show how it works.

In [ ]:
plot = df.hvplot.scatter(x="group", y="value")
pn.pane.HoloViews(plot, backend='bokeh', sizing_mode="stretch_width", height=300)

### Matplotlib

In [ ]:
plot = df.hvplot.scatter(x="group", y="value")
pn.pane.HoloViews(plot, backend='matplotlib', sizing_mode="stretch_width", height=300)

Please note that in a *server context* you might have to set the matplotlib backend like below depending on your setup.

```python
import matplotlib
matplotlib.use('agg')
```

You can make your matplotlib plot stretch responsively by ?? **DON'T KNOW HOW TO. PLEASE HELP**

### Plotly

To use the 'plotly' plotting backend you will need to run `hv.extension("plotly")` to configure the 'plotly' backend.

If you are using `hvPlot` you can use `hvplot.extension("plotly")` instead.

In [ ]:
hvplot.extension("plotly")

plot = df.hvplot.scatter(x="group", y="value", height=300, responsive=True)
pn.pane.HoloViews(plot, backend='plotly', height=300)

**DO NOT WORK!**. This plot is higher than 300px. **DON'T KNOW HOW TO SOLVE**.

Lets change the default backend back to 'bokeh'

In [ ]:
hvplot.extension("bokeh")

### Dynamic

You can change the plotting backend dynamically via a widget too.

In [ ]:
plot = df.hvplot.scatter(x="group", y="value", height=300, responsive=True, title="Try changing the backend")
plot_pane = pn.pane.HoloViews(plot, backend='bokeh', sizing_mode="stretch_width", height=300)
widget = pn.widgets.RadioButtonGroup.from_param(plot_pane.param.backend, button_type="primary", button_style="outline")
pn.Column(widget, plot_pane)

**DOES NOT WORK WELL!**. The matplotlib and plotly plots don't have the title. And they don't stretch their width. And the plotly plot is higher than the 300px specified. **DON'T KNOW HOW TO FIX. PLEASE HELP**

## Linked Axes

By default the axes of plots with shared key or value dimensions are linked. You can remove the link by setting the `linked_axes` parameter to `False`.

In [ ]:
not_linked_plot = df.hvplot.scatter(x="group", y="value", responsive=True, title="Not Linked Axes").opts(active_tools=['box_zoom'])
linked_plot = df.hvplot.scatter(x="group", y="value", responsive=True, title="Linked Axes").opts(active_tools=['box_zoom'])


pn.Column(
    pn.pane.HoloViews(not_linked_plot, backend='bokeh', sizing_mode="stretch_width", height=200, linked_axes=False),
    pn.pane.HoloViews(linked_plot, backend='bokeh', sizing_mode="stretch_width", height=200),
    pn.pane.HoloViews(linked_plot, backend='bokeh', sizing_mode="stretch_width", height=200),
)

## Theme

You can change the `theme`.

In [ ]:
plot = df.hvplot.scatter(x="group", y="value", height=300, responsive=True)
pn.pane.HoloViews(plot, height=300, theme="night_sky")

DON'T KNOW IF THIS WORKS IN JUPYTER. IN VS CODE IT DOES NOT WORK

## Layout and Widget Parameters

Please note that above we used the `Holoviews` pane to display HoloViews objects.

The parameters we explain below works with `pn.panel` and equivalently `pn.pane.HoloViews(...).layout` instead.

### Center

You can center your plots via the `center` parameter.

In [ ]:
plot = df.hvplot.scatter(x="group", y="value", height=300, width=400)

pn.Column(
    "## Not Centered: `center=False`",
    pn.panel(plot, sizing_mode="stretch_width"),
    "## Centered: `center=True`",
    pn.panel(plot, center=True, sizing_mode="stretch_width"),
)

**DO NOT WORK AS I WOULD EXPECT!**. I would have expected the NOT CENTERED plot to have been left aligned and 400px wide.

### HoloMap and DynamicMap with Widgets

[HoloViews](https://holoviews.org/) (the framework) natively renders plots with widgets if a [`HoloMap`](https://holoviews.org/reference/containers/bokeh/HoloMap.html) or [DynamicMap](https://holoviews.org/reference/containers/bokeh/DynamicMap.html) declares any *key dimensions*. This approach efficiently updates just the data inside a plot instead of replacing it entirely.

You can control the layout via the parameters

layout
widgets
widgets
widget_location
widget_layout
widget_type

In [ ]:
import pandas as pd
import hvplot.pandas
import holoviews.plotting.bokeh

def sine(frequency=1.0, amplitude=1.0, function='sin'):
    xs = np.arange(200)/200*20.0
    ys = amplitude*getattr(np, function)(frequency*xs)
    return pd.DataFrame(dict(y=ys), index=xs).hvplot(height=250, responsive=True)

dmap = hv.DynamicMap(sine, kdims=['frequency', 'amplitude', 'function']).redim.range(
    frequency=(0.1, 10), amplitude=(1, 10)).redim.values(function=['sin', 'cos', 'tan'])

pn.panel(dmap, height=500, widget_location="bottom", widget_layout=pn.Row, sizing_mode="stretch_width")

**DOES NOT WORK!**. The `default_layout` is not taken into account. And why is the frequency and amplitude widgets not aligned vertically?

Lets inspect the layout

In [ ]:
print(hv_panel)

You can see that

- the outer `Row` contains the `HoloViews` pane and a `WidgetBox`.
- the 3 widgets corresponding to the 3 *key dimensions* (`kdims`) are laid out in the `WidgetBox`

Lets inspect the `widgets`

In [ ]:
print(hv_panel.widgets)

Lets inspect the `widgets`

In [ ]:
Lets try to change the name of the `FloatSlider`.

In [ ]:
float_slider = hv_panel[1][0]
float_slider.name="Frequency (updated)"
float_slider

Lets try to reorganize the layout

In [ ]:
widgets = hv_panel[1]

pn.Column(hv_panel[0], pn.Column(*widgets))

However, more conveniently the HoloViews pane offers options to lay out the plot and widgets in a number of preconfigured arrangements using the ``center`` and ``widget_location`` parameters.

In [ ]:
pn.panel(dmap, center=True, widget_location='right_bottom')

The ``widget_location`` parameter accepts all of the following options:
    
    ['left', 'bottom', 'right', 'top', 'top_left', 'top_right', 'bottom_left',
     'bottom_right', 'left_top', 'left_bottom', 'right_top', 'right_bottom']

#### Customizing widgets

As we saw above, the HoloViews pane will automatically try to generate appropriate widgets for the type of data, usually defaulting to ``DiscreteSlider`` and ``Select`` widgets. This behavior can be modified by providing a dictionary of ``widgets`` by dimension name.  The values of this dictionary can override the default widget in one of three ways:

* Supplying a ``Widget`` instance
* Supplying a compatible ``Widget`` type
* Supplying a dictionary of ``Widget`` parameter overrides
    
``Widget`` instances will be used as they are supplied and are expected to provide values matching compatible with the values defined on HoloMap/DynamicMap. Similarly if a ``Widget`` type is supplied it should be discrete if the parameter space defines a discrete set of values.  If the defined parameter space is continuous, on the other hand, it may supply any valid value.

In the example below the 'amplitude' dimension is overridden with an explicit ``Widget`` instance, the 'function' dimension is overridden with a RadioButtonGroup letting us toggle between the different functions, and lastly the 'value' parameter on the 'frequency' widget is overridden to change the initial value:

In [ ]:
hv_panel = pn.pane.HoloViews(dmap, widgets={
    'amplitude': pn.widgets.LiteralInput(value=1., type=(float, int)),
    'function': pn.widgets.RadioButtonGroup,
    'frequency': {'value': 5}
}).layout